In [5]:
import pandas as pd

data = pd.read_csv("Combined_News_DJIA.csv", encoding='latin-1')

s = ["Date", "Label"]
for i in range(1, 26):
    s += ["Top" + str(i)]

data_list = []

for i in range(27):
    data_list.append(data[s[i]].tolist())

count = 0
try:
    for i in range(2, 27):
        for j in range(len(data)):
            if type(data_list[i][j]) == float:
                data_list[i][j] = "None"
            else:
                count += 1
                data_list[i][j] = data_list[i][j][2:-1]
except:
    print(i, j,count)

In [26]:
num_features = 25

if (num_features > 1):
    topn = []

    for j in range(len(data)):
      x = ""

      for i in range(2, num_features + 1):
        x += data_list[i][j] + " "
      x += data_list[num_features + 1][j]
      topn.append(x)

else:
     topn = data_list[2]

In [27]:
data_dict = {}
label_dict = {}

for i in range(1, 26):
    data_dict["Top" + str(i)] = data_list[i+1]

label_dict["Label"] = data_list[1]

train_df = pd.DataFrame(data_dict)
label_df = pd.DataFrame(label_dict)

# train_df.head()
# label_df.head()

In [28]:
top1 = [wrd.split() for wrd in topn]
top1[0]

['Georgia',
 "'downs",
 'two',
 'Russian',
 "warplanes'",
 'as',
 'countries',
 'move',
 'to',
 'brink',
 'of',
 'war',
 'BREAKING:',
 'Musharraf',
 'to',
 'be',
 'impeached.',
 'Russia',
 'Today:',
 'Columns',
 'of',
 'troops',
 'roll',
 'into',
 'South',
 'Ossetia;',
 'footage',
 'from',
 'fighting',
 '(YouTube)',
 'Russian',
 'tanks',
 'are',
 'moving',
 'towards',
 'the',
 'capital',
 'of',
 'South',
 'Ossetia,',
 'which',
 'has',
 'reportedly',
 'been',
 'completely',
 'destroyed',
 'by',
 'Georgian',
 'artillery',
 'fire',
 'Afghan',
 'children',
 'raped',
 'with',
 "'impunity,'",
 'U.N.',
 'official',
 'says',
 '-',
 'this',
 'is',
 'sick,',
 'a',
 'three',
 'year',
 'old',
 'was',
 'raped',
 'and',
 'they',
 'do',
 'nothing',
 '150',
 'Russian',
 'tanks',
 'have',
 'entered',
 'South',
 'Ossetia',
 'whilst',
 'Georgia',
 'shoots',
 'down',
 'two',
 'Russian',
 'jets.',
 'Breaking:',
 'Georgia',
 'invades',
 'South',
 'Ossetia,',
 'Russia',
 'warned',
 'it',
 'would',
 'interven

In [29]:
#This code is coppied from the following source:
# https://github.com/dilipvaleti/Binary-Classification-using-word2vect/blob/main/Classification%20using%20word2vect.ipynb

import gensim
import numpy as np

# Train the word2vec model
w2v_model = gensim.models.Word2Vec(top1[:1500],
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [30]:
#This code is coppied from the following source:
# https://github.com/dilipvaleti/Binary-Classification-using-word2vect/blob/main/Classification%20using%20word2vect.ipynb
 (w2v_model.wv['flight']).shape

(100,)

In [31]:
#This code is coppied from the following source:
# https://github.com/dilipvaleti/Binary-Classification-using-word2vect/blob/main/Classification%20using%20word2vect.ipynb

# Generate aggregated sentence vectors based on the word vectors for each word in the sentence
# Replace the words in each text message with the learned word vector
words = set(w2v_model.wv.index_to_key )

main_list = []
main_list_test = []

for news in top1[:1500]:
  data_vec = []
  for word in news:
    if word in words:
      data_vec.append(w2v_model.wv[word])
  main_list.append(np.array(data_vec))

for news in top1[1500:]:
  data_vec = []
  for word in news:
    if word in words:
      data_vec.append(w2v_model.wv[word])
  main_list_test.append(np.array(data_vec))

X_train_vect_avg = []
for v in main_list:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))

X_test_vect_avg = []
for v in main_list_test:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [32]:
#This code is coppied from the following source:
# https://github.com/dilipvaleti/Binary-Classification-using-word2vect/blob/main/Classification%20using%20word2vect.ipynb

# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf_model = rf.fit(np.array(X_train_vect_avg), np.array(data_list[1][:1500]))

In [33]:
#This code is coppied from the following source:
# https://github.com/dilipvaleti/Binary-Classification-using-word2vect/blob/main/Classification%20using%20word2vect.ipynb

# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

y_test = np.array(data_list[1][1500:])

y_pred = rf_model.predict(np.array(X_test_vect_avg))
precision = precision_score(y_test, y_pred)

accuracy = (y_pred==y_test).sum()/len(y_pred)

accuracy

0.45807770961145194

In [ ]:
#This code is coppied from the following source:
# https://inside-machinelearning.com/en/a-simple-and-efficient-model-for-binary-classification-in-nlp/#Create_our_Deep_Learning_model_-_Binary_Classification_NLP

from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(100,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

nn_train = np.array(X_train_vect_avg).astype('float32')
nn_label = np.array(data_list[1][:1500]).astype('float32')

history = model.fit(nn_train,
                    nn_label,
                    epochs=20,
                    batch_size=1)

Epoch 1/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6931 - accuracy: 0.5287
Epoch 2/20
1500/1500 [==============================] - 2s 2ms/step - loss: 0.6930 - accuracy: 0.5327
Epoch 3/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6911 - accuracy: 0.5413
Epoch 4/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6900 - accuracy: 0.5413
Epoch 5/20
1500/1500 [==============================] - 4s 2ms/step - loss: 0.6910 - accuracy: 0.5413
Epoch 6/20
1500/1500 [==============================] - 6s 4ms/step - loss: 0.6907 - accuracy: 0.5413
Epoch 7/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6908 - accuracy: 0.5413
Epoch 8/20
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6904 - accuracy: 0.5413
Epoch 9/20
1500/1500 [==============================] - 2s 2ms/step - loss: 0.6906 - accuracy: 0.5413
Epoch 10/20
1500/1500 [==============================] - 3s 2ms/step - loss: 0.691